In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pandas import DataFrame, read_csv

In [2]:
import geocoder
import folium
from folium.plugins import HeatMap

In [2]:
f = "/Users/samarth/Desktop/Final Year Project/BrazilWeather.csv"
df = read_csv(f)
df.shape

(9779168, 31)

In [193]:
ndf = df
#ndf

In [194]:
ndf = ndf.drop(['wsnm', 'inme','city','prov','date','prcp','stp','smax','smin','gbrd','dewp', 'tmax', 'dmax', 'tmin', 'dmin', 'hmdy', 'hmax', 'hmin', 'wdsp', 'wdct', 'gust'],axis = 1)
ndf = ndf[ndf['yr'] == 2016]
ndf = ndf[ndf['mo'] == 7]
#ndf

In [195]:
ndf[ndf.isnull().any(axis=1)]

,wsid,elvt,lat,lon,mdct,yr,mo,da,hr,temp
3346425,338,1077.0,-21.918066,-46.382996,2016-07-19 09:00:00,2016,7,19,9,NaN
3432341,339,1281.0,-22.314567,-45.373049,2016-07-08 05:00:00,2016,7,8,5,NaN
8241857,403,399.0,-22.372832,-50.974710,2016-07-08 09:00:00,2016,7,8,9,NaN


In [196]:
#Removing the time point from all sensors, when null value occured in any one sensor
nullRows = ['2016-07-19 09:00:00','2016-07-08 05:00:00','2016-07-08 09:00:00']
for i in nullRows:
    ndf = ndf[ndf['mdct'] != i]
    

In [197]:
ndf.shape

(90402, 10)

In [199]:
valMat = []
locMat = []
for i in set(ndf['wsid']):
    a = ndf[ndf['wsid'] == i]
    b = a.iloc[0][['lat','lon']]
    #print("\n")
    valMat.append(a['temp'])
    locMat.append(b)
    
valMat = np.array(valMat)
locMat = np.array(locMat)
valMat = np.transpose(valMat)
print(locMat.shape)
print(valMat.shape)
#print(np.transpose(locMat))

(122, 2)
(741, 122)





Proceed As Aakash's PCA code from here 

In [200]:
valMat[:,1]

array([ 22.3,  22. ,  22.2,  22. ,  21.7,  21.5,  21.3,  20.6,  20. ,
        19.4,  20.4,  22.2,  22.5,  23.4,  24.1,  24.9,  25.2,  26. ,
        25.6,  24.4,  24. ,  23.6,  23.2,  22.8,  21.6,  20.9,  20.9,
        21.4,  21.4,  21.3,  21.2,  20.2,  19.5,  19.2,  19.5,  21.9,
        23.3,  23.3,  24.2,  25. ,  25.2,  25.3,  25.1,  24.8,  23.7,
        22.9,  21.4,  20.7,  20.9,  21.4,  21.3,  21.1,  20.5,  20. ,
        19.9,  19.8,  18.8,  18.4,  19.1,  20.3,  22.7,  22.3,  23.3,
        24.7,  24.3,  25.1,  24.7,  24.3,  23.4,  22.5,  20.7,  20. ,
        20.4,  21. ,  20.4,  20.3,  20.2,  20.5,  20.2,  20.2,  20.1,
        20.2,  20.4,  20.9,  21.7,  22.9,  22.9,  23.4,  24.7,  24.8,
        25.6,  25. ,  24.1,  23.3,  22.2,  21.3,  21.2,  20.1,  19.7,
        19.8,  19.5,  19.8,  19.4,  18.7,  18.6,  18.4,  18.6,  20. ,
        21.2,  24. ,  25.2,  26. ,  26.6,  26.5,  26.1,  25.1,  24.3,
        23.9,  22.6,  21.7,  21. ,  19.9,  19.4,  19.2,  19.1,  19.2,
        20. ,  20.3,

In [201]:
meanVal = np.mean(valMat,1)
meanVal

array([ 17.10655738,  16.55491803,  16.11311475,  15.76967213,
        15.41147541,  15.19672131,  14.95901639,  14.76639344,
        14.40245902,  14.35491803,  14.57786885,  16.38770492,
        18.2795082 ,  20.15819672,  21.80655738,  22.93688525,
        23.27459016,  23.33770492,  23.52704918,  23.07622951,
        21.63278689,  19.74754098,  18.74180328,  17.89590164,
        17.54016393,  16.90081967,  16.29508197,  15.9442623 ,
        15.55737705,  15.37622951,  14.95901639,  14.70983607,
        14.48196721,  14.29590164,  14.34508197,  16.49590164,
        18.69180328,  20.53606557,  22.16803279,  23.26229508,
        23.90737705,  23.95901639,  24.03032787,  23.51639344,
        21.95491803,  19.72459016,  18.78852459,  17.81065574,
        17.06885246,  16.37622951,  15.91639344,  15.41639344,
        15.04672131,  14.72704918,  14.37868852,  14.24508197,
        13.91065574,  13.71393443,  13.75409836,  16.04672131,
        18.46147541,  20.46803279,  22.10983607,  23.19

In [202]:
stdVal = np.std(valMat,1)
stdVal

array([ 5.16526447,  5.27757555,  5.22170676,  5.14794069,  5.05935583,
        5.09300282,  5.06402863,  5.00069849,  5.14509894,  5.02798892,
        5.0772316 ,  5.09319021,  5.2252216 ,  5.30721501,  5.23673559,
        5.07173461,  5.65541321,  6.14644143,  5.74336448,  5.61048569,
        5.75044464,  5.29492818,  5.07586446,  5.1784452 ,  4.92144288,
        5.16224465,  5.25082121,  5.15995058,  5.08103059,  5.14711527,
        4.99304031,  4.99813752,  4.95403489,  4.91600593,  5.04497969,
        4.8001434 ,  5.05803354,  5.36624339,  5.34067698,  5.50164398,
        5.6884578 ,  5.70808679,  5.81055977,  5.74840478,  5.8537098 ,
        5.64092644,  5.17467345,  5.35167826,  5.19582244,  5.30040291,
        5.18922845,  5.06488005,  4.95251525,  4.95492861,  4.81585887,
        4.81704798,  4.72228424,  4.63437469,  4.78611632,  4.63935737,
        4.9317547 ,  5.24079625,  5.18004383,  5.42976019,  5.57515249,
        5.69829652,  5.69506191,  5.70557326,  5.58388986,  5.42

In [203]:
print("Final Mean: %f and Std: %f"%(np.mean(meanVal),np.mean(stdVal)))

Final Mean: 18.048419 and Std: 5.879669


In [208]:
map = folium.Map(location=[locMat[1][0],locMat[1][1]], zoom_start=3)
for point in range(0, len(locMat)):
    #folium.Marker(locMat[point]).add_to(map)
    folium.CircleMarker(locMat[point], radius=3, color='red').add_to(map)
map

In [209]:
hmap = folium.Map(location=[locMat[1][0],locMat[1][1]], zoom_start=3, )
values = [[locMat[i,0],locMat[i,1],valMat[0,i]] for i in range(122)]
hm = HeatMap(values)
hmap.add_child(hm)
hmap

In [210]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
reducedMat = pca.fit_transform(valMat)

In [211]:
np.mean(np.mean(reducedMat,1) + 40)

40.0

In [212]:
np.mean(np.std(reducedMat,1))

13.1967631738191

In [213]:
pca.explained_variance_ratio_

array([ 0.67967132,  0.07842561,  0.06536804,  0.03101496,  0.02744797,
        0.01942801,  0.01158551,  0.009263  ,  0.00738071,  0.00470809,
        0.00440451,  0.00376253,  0.00326692,  0.00286235,  0.00266657])

In [214]:
rMeanVal = np.mean(pca.components_,1)
rMeanVal

array([ 0.08476055, -0.00898619,  0.00211476,  0.00683885,  0.01137938,
       -0.00674432, -0.00639168,  0.00576088,  0.00344563, -0.00683113,
        0.00783223, -0.0008457 , -0.00645975,  0.00153871,  0.00290438])

In [218]:
xyz = abs(pca.components_)
indices = np.argmax(xyz,1)
print(indices)
reducedData = []
valMat_T = np.transpose(valMat)
for i in indices:
    reducedData.append(list(valMat_T[i,:]))
reducedData = np.transpose(np.array(reducedData))
rMean = np.mean(reducedData,1)
rStd = np.std(reducedData,1)
print("Reduced mean: %f and std: %f"%(np.mean(rMean),np.mean(rStd)))

[ 54  51  54  54  54  59  93  36  36  36  59 117  36 107  93]
Reduced mean: 12.385173 and std: 7.734827


In [221]:
print(reducedData)

[[  0.   17.6   0.  ...,  11.5  20.3  21.7]
 [  0.   18.7   0.  ...,  10.6  19.9   0. ]
 [  0.   19.4   0.  ...,  10.3   0.    0. ]
 ..., 
 [  0.    0.    0.  ...,  18.5  24.5  26.1]
 [  0.    0.    0.  ...,  17.7  23.2  24.5]
 [  0.    0.    0.  ...,  17.1  21.9  23.7]]


In [216]:
reducedData.shape

(741, 15)

In [217]:
map = folium.Map(location=[locMat[1][0],locMat[1][1]], zoom_start=3)
for point in indices:
    #folium.Marker(locMat[point]).add_to(map)
    folium.CircleMarker(locMat[point], radius=3, color='red').add_to(map)
map

In [227]:
rhmap = folium.Map(location=[locMat[1][0],locMat[1][1]], zoom_start=3, )
k = 0
rvalues = []
for i in indices:
        rvalues.append([locMat[i,0],locMat[i,1],reducedData[0,k]])
        k = k+1
#rvalues = [[locMat[i,0],locMat[i,1],reducedData[0,k]] for i in indices k = k + 1]
rhm = HeatMap(rvalues)
rhmap.add_child(rhm)
rhmap